## This code performs whole-brain simulations where GRNs of different types are activated and the activity of all other neurons is recorded.

### Simulations are based on the leaky integrate and fire model by Shiu et al. (bioRxiv, 2023). 

### This notebook is adapted from one generously shared by Philip Shiu et al. (https://github.com/philshiu/Drosophila_brain_model)

In [19]:
from model import run_exp
from model import default_params as params
import utils as utl
from brian2 import Hz

config = {
    'path_res'  : './results/connectome_paper',                     # directory to store results
    'path_comp' : './2023_03_23_completeness_630_final.csv',        # csv of the complete list of Flywire neurons
    'path_con'  : './2023_03_23_connectivity_630_final.parquet',    # connectivity data
    'n_proc'    : -1,                                               # number of CPU cores (-1: use all)
}

## Underlying connectivity data
The connectivity of the fly brain is stored in the folowing files:
- neurons present: `config['path_comp']`
- connectivity between neurons: `config['path_con]`

## Model parameters
The equation and constants for the leaky integrate and fire model are defined 
in the dictionary `default_params` in the beginning of the file `model.py`:

```
default_params = {
    # trials
    't_run'     : 1000 * ms,              # duration of trial
    'n_run'     : 30,                     # number of runs

    'v_0'       : -52 * mV,               # resting potential
    'v_rst'     : -52 * mV,               # reset potential after spike
    [...]
```
We can also change values
and pass the modified dictionary to the model (see Experiment 1).

## First, define each set of GRNs.
## We are using the same neurons as for the connectome analyses.

In [20]:
# Sugar GRNs in left hemisphere
neu_sugar_L = [
    720575940610788069,
    720575940611875570,
    720575940612670570,
    720575940613601698,
    720575940616885538,
    720575940617000768,
    720575940620900446,
    720575940621502051,
    720575940621754367,
    720575940624963786,
    720575940628853239,
    720575940629176663,
    720575940630233916,
    720575940630797113,
    720575940632425919,
    720575940632889389,
    720575940633143833,
    720575940637568838,
    720575940638202345,
    720575940639198653,
    720575940639332736,
    720575940640649691,
    ]

# Bitter GRNs in left hemisphere
neu_bitter_L = [
    720575940602353632,
    720575940604027168,
    720575940610259370,
    720575940610481370,
    720575940610483162,
    720575940613061118,
    720575940614281266,
    720575940617094208,
    720575940618600651,
    720575940619028208,
    720575940619197093,
    720575940621008895,
    720575940621778381,
    720575940622298631,
    720575940626287336,
    720575940627578156,
    720575940629146711,
    720575940630195909,
    720575940645743412,
    720575940646212996,
    ]

# Water GRNs in left hemisphere
neu_water_L = [
    720575940606002609,
    720575940612579053,
    720575940612950568,
    720575940613786774,
    720575940613996959,
    720575940616177458,
    720575940617857694,
    720575940622486922,
    720575940622902535,
    720575940625203504,
    720575940625861168,
    720575940629852866,
    720575940630553415,
    720575940631898285,
    720575940634796536,
    720575940635172191,
    720575940644965399,
    720575940660292225,
    ]

# IR94e GRNs in left hemisphere
neu_IR94e_L = [
    720575940610683315,
    720575940612920386,
    720575940614211295,
    720575940621375231,
    720575940624079544,
    720575940626016017,
    720575940628198503,
    720575940631082124,
    720575940638218173,
    ]

For an easier identification, we define also a mapping from the flywire IDs to custom names. The above neurons are calles sugar_1, sugar_2 etc:

In [21]:
flyid2name_sugar = { f: 'sugar_L_{}'.format(i+1) for i, f in enumerate(neu_sugar_L) }
flyid2name_bitter = { f: 'bitter_L_{}'.format(i+1) for i, f in enumerate(neu_bitter_L) }
flyid2name_water = { f: 'water_L_{}'.format(i+1) for i, f in enumerate(neu_water_L) }
flyid2name_IR94e = { f: 'IR94e_L_{}'.format(i+1) for i, f in enumerate(neu_IR94e_L) }

# view example
flyid2name_IR94e

{720575940610683315: 'IR94e_L_1',
 720575940612920386: 'IR94e_L_2',
 720575940614211295: 'IR94e_L_3',
 720575940621375231: 'IR94e_L_4',
 720575940624079544: 'IR94e_L_5',
 720575940626016017: 'IR94e_L_6',
 720575940628198503: 'IR94e_L_7',
 720575940631082124: 'IR94e_L_8',
 720575940638218173: 'IR94e_L_9'}

# Running simulations
## Background info:
To run a simulation exciting these nerons we have to call `run_exp` supplying the following:
- unique name for the simulation: `exp_name`
- a list of neurons we want to stimulate: `neu_sugar`
- the connectivity data: `config['path_comp']` and `config['path_con]`
- path to store the output: `config['path_res']`
- number of CPU cores use: `config['n_procs]`

The `.parquet` file created during a simulation contains all spikes events of all neurons in the model.
We load the data again from disk by passing a list of result files to the `utl.load_exps` function.

The spike times can be converted to spike rates [Hz] via `utl.get_rate`, which requires the duration of each trial.
`utl.get_rate` returns `pandas.DataFrame` objects:
1. spike rate for each neuron (rows) in each experiment (column): `df_rate`
2. standard deviation of rate across trials: `df_rate_std`

For convenience, we can optionally pass the `flyid2name` dictionary to `utl.get_rate` in order to convert flywire IDs into
meaningful names.

In [22]:
#show default params
params

{'t_run': 1. * second,
 'n_run': 30,
 'v_0': -52. * mvolt,
 'v_rst': -52. * mvolt,
 'v_th': -45. * mvolt,
 't_mbr': 20. * msecond,
 'tau': 5. * msecond,
 't_rfc': 2.2 * msecond,
 't_dly': 1.8 * msecond,
 'w_syn': 275. * uvolt,
 'r_poi': 150. * hertz,
 'r_poi2': 0. * hertz,
 'f_poi': 250,
 'eqs': '\ndv/dt = (v_0 - v + g) / t_mbr : volt (unless refractory)\ndg/dt = -g / tau               : volt (unless refractory) \nrfc                            : second\n',
 'eq_th': 'v > v_th',
 'eq_rst': 'v = v_rst; w = 0; g = 0 * mV'}

## Sugar GRN activation

In [41]:
# Run simulation at diff stim intensities

for stim_rate in [25,50,75,100,125,150,175,200]: 
    
    prefix = 'sugarGRN_' + str(stim_rate) + 'Hz'
    params['r_poi'] = stim_rate * Hz
    run_exp(exp_name=prefix, neu_exc=neu_sugar_L, params=params, **config)
    
    # extract results
    datafilename = './results/connectome_paper/' + prefix + '.parquet'
    df_spike = utl.load_exps([datafilename])
    df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name_sugar)
    
    # save dataframes to csv
    savepath = 'results/connectome_paper/'
    df_rate.fillna(0).to_csv(savepath + prefix + '_rates.csv')
    df_rate_std.fillna(0).to_csv(savepath + prefix + '_std.csv')

>>> Skipping experiment sugarGRN_25Hz because results/connectome_paper/sugarGRN_25Hz.parquet exists and force_overwrite = False
>>> Skipping experiment sugarGRN_50Hz because results/connectome_paper/sugarGRN_50Hz.parquet exists and force_overwrite = False
>>> Experiment:     sugarGRN_75Hz
    Output file:    results/connectome_paper/sugarGRN_75Hz.parquet
    Excited neurons: 22


INFO       Cache size for target 'cython': 1056387166 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the '/Users/anitadevineni/Library/Caches/cython/brian_extensions' directory. [brian2]
INFO       Cache size for target 'cython': 1056387166 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the '/Users/anitadevineni/Library/Caches/cython/brian_extensions' directory. [brian2]
INFO       Cache size for target 'cython': 1056387166 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the '/Users/anitadevineni/Library/Caches/cython/brian_extensions' directory. [brian2]
INFO       Cache size for target 'cython': 1056387166 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the '/Users/anitadevineni/Library/Caches/cython/brian_extensions' directory. [brian2]
INFO       Cache size for target

    Elapsed time:   299 s


WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


>>> Experiment:     sugarGRN_100Hz
    Output file:    results/connectome_paper/sugarGRN_100Hz.parquet
    Excited neurons: 22
    Elapsed time:   268 s


...

WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


>>> Experiment:     sugarGRN_175Hz
    Output file:    results/connectome_paper/sugarGRN_175Hz.parquet
    Excited neurons: 22
    Elapsed time:   295 s


WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


>>> Experiment:     sugarGRN_200Hz
    Output file:    results/connectome_paper/sugarGRN_200Hz.parquet
    Excited neurons: 22
    Elapsed time:   200 s


WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


## Bitter GRN activation

In [42]:
# Run simulation at diff stim intensities

for stim_rate in [25,50,75,100,125,150,175,200]: 
    
    prefix = 'bitterGRN_' + str(stim_rate) + 'Hz'
    params['r_poi'] = stim_rate * Hz
    run_exp(exp_name=prefix, neu_exc=neu_bitter_L, params=params, **config)
    
    # extract results
    datafilename = './results/connectome_paper/' + prefix + '.parquet'
    df_spike = utl.load_exps([datafilename])
    df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name_bitter)
    
    # save dataframes to csv
    savepath = 'results/connectome_paper/'
    df_rate.fillna(0).to_csv(savepath + prefix + '_rates.csv')
    df_rate_std.fillna(0).to_csv(savepath + prefix + '_std.csv')

>>> Experiment:     bitterGRN_25Hz
    Output file:    results/connectome_paper/bitterGRN_25Hz.parquet
    Excited neurons: 20


WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


    Elapsed time:   268 s
>>> Experiment:     bitterGRN_50Hz
    Output file:    results/connectome_paper/bitterGRN_50Hz.parquet
    Excited neurons: 20
    Elapsed time:   319 s


WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


>>> Experiment:     bitterGRN_75Hz
    Output file:    results/connectome_paper/bitterGRN_75Hz.parquet
    Excited neurons: 20
    Elapsed time:   329 s


...

WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


>>> Experiment:     bitterGRN_175Hz
    Output file:    results/connectome_paper/bitterGRN_175Hz.parquet
    Excited neurons: 20
    Elapsed time:   376 s


WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


>>> Experiment:     bitterGRN_200Hz
    Output file:    results/connectome_paper/bitterGRN_200Hz.parquet
    Excited neurons: 20
    Elapsed time:   325 s


WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


## Water GRN activation

In [43]:
# Run simulation at diff stim intensities

for stim_rate in [25,50,75,100,125,150,175,200]: 
    
    prefix = 'waterGRN_' + str(stim_rate) + 'Hz'
    params['r_poi'] = stim_rate * Hz
    run_exp(exp_name=prefix, neu_exc=neu_water_L, params=params, **config)
    
    # extract results
    datafilename = './results/connectome_paper/' + prefix + '.parquet'
    df_spike = utl.load_exps([datafilename])
    df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name_water)
    
    # save dataframes to csv
    savepath = 'results/connectome_paper/'
    df_rate.fillna(0).to_csv(savepath + prefix + '_rates.csv')
    df_rate_std.fillna(0).to_csv(savepath + prefix + '_std.csv')

>>> Experiment:     waterGRN_25Hz
    Output file:    results/connectome_paper/waterGRN_25Hz.parquet
    Excited neurons: 18
    Elapsed time:   358 s


WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


>>> Experiment:     waterGRN_50Hz
    Output file:    results/connectome_paper/waterGRN_50Hz.parquet
    Excited neurons: 18
    Elapsed time:   179 s


WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


>>> Experiment:     waterGRN_75Hz
    Output file:    results/connectome_paper/waterGRN_75Hz.parquet
    Excited neurons: 18
    Elapsed time:   354 s


...

WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


>>> Experiment:     waterGRN_175Hz
    Output file:    results/connectome_paper/waterGRN_175Hz.parquet
    Excited neurons: 18
    Elapsed time:   294 s


WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


>>> Experiment:     waterGRN_200Hz
    Output file:    results/connectome_paper/waterGRN_200Hz.parquet
    Excited neurons: 18
    Elapsed time:   377 s


WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


## IR94e GRN activation

In [45]:
# Run simulation at diff stim intensities

for stim_rate in [25,50,75,100,125,150,175,200]: 
    
    prefix = 'IR94eGRN_' + str(stim_rate) + 'Hz'
    params['r_poi'] = stim_rate * Hz
    run_exp(exp_name=prefix, neu_exc=neu_IR94e_L, params=params, **config)
    
    # extract results
    datafilename = './results/connectome_paper/' + prefix + '.parquet'
    df_spike = utl.load_exps([datafilename])
    df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name_IR94e)
    
    # save dataframes to csv
    savepath = 'results/connectome_paper/'
    df_rate.fillna(0).to_csv(savepath + prefix + '_rates.csv')
    df_rate_std.fillna(0).to_csv(savepath + prefix + '_std.csv')

>>> Skipping experiment IR94eGRN_25Hz because results/connectome_paper/IR94eGRN_25Hz.parquet exists and force_overwrite = False
>>> Skipping experiment IR94eGRN_50Hz because results/connectome_paper/IR94eGRN_50Hz.parquet exists and force_overwrite = False
>>> Skipping experiment IR94eGRN_75Hz because results/connectome_paper/IR94eGRN_75Hz.parquet exists and force_overwrite = False
>>> Skipping experiment IR94eGRN_100Hz because results/connectome_paper/IR94eGRN_100Hz.parquet exists and force_overwrite = False
>>> Skipping experiment IR94eGRN_125Hz because results/connectome_paper/IR94eGRN_125Hz.parquet exists and force_overwrite = False
>>> Experiment:     IR94eGRN_150Hz
    Output file:    results/connectome_paper/IR94eGRN_150Hz.parquet
    Excited neurons: 9


INFO       Cache size for target 'cython': 1294011045 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the '/Users/anitadevineni/Library/Caches/cython/brian_extensions' directory. [brian2]
INFO       Cache size for target 'cython': 1294011045 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the '/Users/anitadevineni/Library/Caches/cython/brian_extensions' directory. [brian2]
INFO       Cache size for target 'cython': 1294011045 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the '/Users/anitadevineni/Library/Caches/cython/brian_extensions' directory. [brian2]
INFO       Cache size for target 'cython': 1294011045 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the '/Users/anitadevineni/Library/Caches/cython/brian_extensions' directory. [brian2]
INFO       Cache size for target

    Elapsed time:   211 s


WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


>>> Experiment:     IR94eGRN_175Hz
    Output file:    results/connectome_paper/IR94eGRN_175Hz.parquet
    Excited neurons: 9
    Elapsed time:   199 s


WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]


>>> Experiment:     IR94eGRN_200Hz
    Output file:    results/connectome_paper/IR94eGRN_200Hz.parquet
    Excited neurons: 9
    Elapsed time:   191 s


WARNING    /Users/anitadevineni/opt/anaconda3/envs/brian2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 [py.warnings]
